In [29]:
import time
from datetime import datetime, timedelta
import vk_api
import queue
import gspread
from oauth2client.service_account import ServiceAccountCredentials


def get_group_ids(spreadsheet_key, sheet_name, range_name):
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(credentials)
    spreadsheet = client.open_by_key(spreadsheet_key)
    group_ids_range = spreadsheet.worksheet(sheet_name).range(range_name)
    group_ids = queue.Queue()
    for cell in group_ids_range:
        if cell.value:
            group_ids.put('-' + cell.value)
    return group_ids


def get_or_create_worksheet(spreadsheet, worksheet_name):
    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="100", cols="30")  # Увеличил количество столбцов
    return worksheet


def clear_worksheet(worksheet):
    # Очищение и удаление всех строк, кроме первой
    all_rows = worksheet.get_all_values()
    num_rows = len(all_rows)
    if num_rows > 1:
        worksheet.delete_rows(2, num_rows - 1)

    # Обновление первой строки с заголовками
    worksheet.update('A1:H1', [['Дата и время проверки', 'Дата публикации', 'Ссылка на пост', 'Текст', 'Комментарии',
                                'Лайки', 'Репосты', 'Весь охват']])

def collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, token):
    vk_session = vk_api.VkApi(token=token)
    api = vk_session.get_api()

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(credentials)
    target_spreadsheet = client.open_by_key(target_spreadsheet_key)

    worksheet_name = group_id
    worksheet = get_or_create_worksheet(target_spreadsheet, worksheet_name)
    clear_worksheet(worksheet)  # Очистка таблицы перед записью новой статистики

    # Вычисление даты начала периода
    start_date = datetime.now().date() - timedelta(days=num_days)

    posts = api.wall.get(owner_id=group_id, count=200, extended=1)['items']
    if posts:
        sorted_posts = sorted(posts, key=lambda x: x['date'], reverse=False)  # Сортировка по дате
        post_count = 0  # Счетчик собранных постов
        rows_to_append = []  # Список строк для добавления

        for post in sorted_posts:
            post_date = datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d %H:%M:%S')

            # Проверка, принадлежит ли пост к периоду
            if datetime.strptime(post_date, '%Y-%m-%d %H:%M:%S').date() >= start_date:
                post_link = f"https://vk.com/wall-{group_id[1:]}_{post['id']}"
                text = post['text']
                comments = post.get('comments', {}).get('count', 0)
                likes = post.get('likes', {}).get('count', 0)
                reposts = post.get('reposts', {}).get('count', 0)
                views = post.get('views', {}).get('count', 0)

                data = [datetime.now().strftime('%Y-%m-%d %H:%M:%S'), post_date, post_link, text, comments,
                        likes, reposts, views]

                rows_to_append.append(data)
                print(f'[{datetime.now()}] Собрано: {post_date} - {post_link}')

                post_count += 1  # Увеличение счетчика собранных постов

                # Проверка количества собранных постов
                if post_count >= min_post_count:
                    break

        # Добавление всех строк в таблицу одновременно
        if rows_to_append:
            worksheet.append_rows(rows_to_append)
            print(f'[{datetime.now()}] Записано {len(rows_to_append)} строк в таблицу.')
        else:
            print(f'[{datetime.now()}] Нет данных для записи в таблицу.')




if __name__ == "__main__":
    spreadsheet_key = "1UEmvKWPyJ2nBOt9VdbCAANxEvFLMddu_Uy75N40Ekzw"
    sheet_name = "Фотографы"
    range_name = "l3:l10"
    target_spreadsheet_key = "1f97AzhN-YFCofuIeA80ZdDGNRJUajW6o8EBN4XFezGE"
    num_days = 3650
    min_post_count = 10000

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('CREDENTIALS.json', scope)
    client = gspread.authorize(credentials)
    spreadsheet_key2 = "1f97AzhN-YFCofuIeA80ZdDGNRJUajW6o8EBN4XFezGE"
    spreadsheet = client.open_by_key(spreadsheet_key2)
    token_sheet = spreadsheet.worksheet('Token')
    token = token_sheet.acell('b1').value
    group_ids = get_group_ids(spreadsheet_key, sheet_name, range_name)

    for _ in range(group_ids.qsize()):  # Обход всех сообществ в очереди
        group_id = group_ids.get()  # Получение следующего сообщества из очереди
        group_ids.put(group_id)  # Возвращение сообщества в конец очереди

        collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, token)


[2023-07-01 11:58:21.858207] Собрано: 2017-01-13 18:20:48 - https://vk.com/wall-35217360_850
[2023-07-01 11:58:22.859181] Собрано: 2017-01-16 15:55:17 - https://vk.com/wall-35217360_853
[2023-07-01 11:58:23.859916] Собрано: 2017-01-16 19:15:04 - https://vk.com/wall-35217360_854
[2023-07-01 11:58:24.860882] Собрано: 2017-01-23 01:27:30 - https://vk.com/wall-35217360_855
[2023-07-01 11:58:25.861184] Собрано: 2017-01-24 16:11:41 - https://vk.com/wall-35217360_856
[2023-07-01 11:58:26.861762] Собрано: 2017-01-25 23:21:43 - https://vk.com/wall-35217360_857
[2023-07-01 11:58:27.862316] Собрано: 2017-01-25 23:28:26 - https://vk.com/wall-35217360_858
[2023-07-01 11:58:28.862835] Собрано: 2017-01-25 23:49:39 - https://vk.com/wall-35217360_859
[2023-07-01 11:58:29.863382] Собрано: 2017-01-29 20:23:33 - https://vk.com/wall-35217360_861
[2023-07-01 11:58:30.864005] Собрано: 2017-02-06 16:01:49 - https://vk.com/wall-35217360_863
[2023-07-01 11:58:31.864727] Собрано: 2017-02-06 16:52:02 - https://vk